<a href="https://colab.research.google.com/github/MiniduTH/BuildMasterWEB/blob/main/BulkDataExtractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This code can use to efficiently extract and process large volumes of data in batches using parallel HTTP requests and HTML parsing.

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import time

# Function to make a POST request and return the result
def fetch_result(index_number):
    # url = "replace here with the url of the site"
    payload = {
        # replace the payload of the post re
    }
    try:
        response = requests.post(url, data=payload)
        return {
            # add attributes of the objects of payload
            # like"indexNumber": index_number
        }
    except requests.RequestException as e:
        return {
            "indexNumber": index_number,
            "status_code": "Error",
            "response": str(e)
        }

# Function to extract data from HTML response
def extract_data_from_html(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')

    # Check if the "Index Number" text is present before proceeding
    index_number_tag = soup.find(text="Index Number")
    if index_number_tag is None:
        return None  # Return None if "Index Number" is not found

    # Extract each field from the HTML content
    examination = index_number_tag.find_next('td').find_next('td').text.strip()
    # replace with the fields you want to extract and save it in a var


    exam_grade = results  # Assuming the Exam Grade is the same as the result

    return [
        #return whatever you saved in vars
    ]

# Parallel fetching results
# because we have to check a large range of data
def fetch_all_results(start, end, batch_size=5000, max_workers=500):
    all_results = []
    for batch_start in range(start, end, batch_size):
        batch_end = min(batch_start + batch_size, end)
        results = []
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            futures = {executor.submit(fetch_result, i): i for i in range(batch_start, batch_end)}
            for future in tqdm(as_completed(futures), total=len(futures), desc=f"Fetching batch {batch_start}-{batch_end}"):
                try:
                    result = future.result()
                    results.append(result)
                except Exception as e:
                    results.append({
                        "indexNumber": futures[future],
                        "status_code": "Error",
                        "response": str(e)
                    })
        all_results.extend(results)
        # Save progress after each batch
        df = pd.DataFrame(results)
        df.to_excel(f"results_{batch_start}_{batch_end}.xlsx", index=False)
    return all_results

# Main function to run the process
def main():
    start_time = time.time()

    # start_index = x
    # end_index = y  # Adjust with the inputs you want to check
    results = fetch_all_results(start_index, end_index, batch_size=5000, max_workers=50)  # Adjust max_workers based on your system

    # Save all results to a single file
    df = pd.DataFrame(results)
    df.to_excel("final_results.xlsx", index=False)

    # Define the desired headers for the new DataFrame
    headers = [
        "x", "y"]  # Replace with the actual headers you want to extract

    # Create an empty list to store the extracted data
    data = []

    # Loop through each row in the DataFrame and extract data
    for _, row in tqdm(df.iterrows(), total=df.shape[0], desc="Extracting data"):
        html_content = row['response']
        extracted_data = extract_data_from_html(html_content)
        if extracted_data is not None:  # Append only if data was extracted
            data.append(extracted_data)

    # Create a new DataFrame with the extracted data
    extracted_df = pd.DataFrame(data, columns=headers)
    extracted_df.to_excel('extracted_results.xlsx', index=False)


#below code is added just to know how much extracted during a timeperiod. this is not necessry to run

    print(f"Data extraction complete. Saved to 'extracted_results.xlsx'.")
    print(f"Elapsed time: {time.time() - start_time} seconds")

if __name__ == "__main__":
    main()

Run below code to convert excel to a  JSON file

In [ ]:
import pandas as pd

df = pd.read_excel('extracted_results.xlsx')

json_data = df.to_json(orient='records')

with open('output.json', 'w') as f:
    f.write(json_data)